In [109]:
import pandas as pd
import json 

In [110]:
Sales_data  = pd.read_json('Sales.json')
forecast_data = pd.read_json('forecast.json')
print(Sales_data.head)
print(forecast_data.head)

<bound method NDFrame.head of         ProductKey                                       Product Name  \
0             2492  Cigarette Lighter Adapter for Contoso Phones E...   
1             2492  Cigarette Lighter Adapter for Contoso Phones E...   
2             2492  Cigarette Lighter Adapter for Contoso Phones E...   
3             2492  Cigarette Lighter Adapter for Contoso Phones E...   
4             2492  Cigarette Lighter Adapter for Contoso Phones E...   
...            ...                                                ...   
298241        1383  Contoso Dual Handset Cordless Phone System E20...   
298242        1383  Contoso Dual Handset Cordless Phone System E20...   
298243        2197              Adventure Works Desk Lamp E1200 Black   
298244        2197              Adventure Works Desk Lamp E1200 Black   
298245         526                              WWI LCD17W E200 Black   

                       Brand                    Color  \
0                    Contoso  Cell p

In [111]:
#print(Sales_data.info())
#print(forecast_data.info())
print("nulls:",Sales_data.isna().sum())
print("duplicated :" ,Sales_data.duplicated().sum())

nulls: ProductKey            0
Product Name          0
Brand                 0
Color                 0
Subcategory           0
Category              0
CustomerKey           0
Customer Code         0
Name             268449
Education        268449
Occupation       268449
Continent             0
City                  0
State                 0
CountryRegion         0
OrderDate             0
Quantity              0
Net Price             0
dtype: int64
duplicated : 218008


In [112]:
Sales_data = Sales_data.drop_duplicates()
print("duplicated :" ,Sales_data.duplicated().sum())
print(Sales_data.info())

duplicated : 0
<class 'pandas.core.frame.DataFrame'>
Index: 80238 entries, 0 to 298245
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ProductKey     80238 non-null  int64  
 1   Product Name   80238 non-null  object 
 2   Brand          80238 non-null  object 
 3   Color          80238 non-null  object 
 4   Subcategory    80238 non-null  object 
 5   Category       80238 non-null  object 
 6   CustomerKey    80238 non-null  int64  
 7   Customer Code  80238 non-null  object 
 8   Name           29797 non-null  object 
 9   Education      29797 non-null  object 
 10  Occupation     29797 non-null  object 
 11  Continent      80238 non-null  object 
 12  City           80238 non-null  object 
 13  State          80238 non-null  object 
 14  CountryRegion  80238 non-null  object 
 15  OrderDate      80238 non-null  object 
 16  Quantity       80238 non-null  int64  
 17  Net Price      80238 non-null  float64


In [113]:
# Fill missing categorical data with 'Unknown'
cols_to_fix = ['Name', 'Education', 'Occupation']
for col in cols_to_fix:
    Sales_data[col] = Sales_data[col].fillna('Unknown')

In [114]:
# there are too many nulls
# best appraoch to check if customerKey has in one raw name, education or occupation val , then add this to same customer key with null values
cols = ['Name', 'Education', 'Occupation']

uniqueness_check = (
    Sales_data.groupby('CustomerKey')[cols]
      .nunique(dropna=True)
)
print(uniqueness_check)

             Name  Education  Occupation
CustomerKey                             
13              1          1           1
14              1          1           1
15              1          1           1
16              1          1           1
17              1          1           1
...           ...        ...         ...
19120           1          1           1
19125           1          1           1
19126           1          1           1
19137           1          1           1
19143           1          1           1

[8868 rows x 3 columns]


In [115]:
conflicting_customers = uniqueness_check[
    (uniqueness_check['Name'] > 1) |
    (uniqueness_check['Education'] > 1) |
    (uniqueness_check['Occupation'] > 1)
]

conflicting_customers.head()

,Name,Education,Occupation
CustomerKey,,,


In [116]:
#correct data types

Sales_data['OrderDate']=pd.to_datetime(Sales_data['OrderDate'],errors='coerce')
str_cols = Sales_data.select_dtypes(include='object').columns
for col in str_cols:
    Sales_data[col]= Sales_data[col].astype('string')

Sales_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 80238 entries, 0 to 298245
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ProductKey     80238 non-null  int64         
 1   Product Name   80238 non-null  string        
 2   Brand          80238 non-null  string        
 3   Color          80238 non-null  string        
 4   Subcategory    80238 non-null  string        
 5   Category       80238 non-null  string        
 6   CustomerKey    80238 non-null  int64         
 7   Customer Code  80238 non-null  string        
 8   Name           80238 non-null  string        
 9   Education      80238 non-null  string        
 10  Occupation     80238 non-null  string        
 11  Continent      80238 non-null  string        
 12  City           80238 non-null  string        
 13  State          80238 non-null  string        
 14  CountryRegion  80238 non-null  string        
 15  OrderDate      80238 no

In [117]:
Sales_data[str_cols] = Sales_data[str_cols].apply(lambda col : col.str.strip())
Sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80238 entries, 0 to 298245
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ProductKey     80238 non-null  int64         
 1   Product Name   80238 non-null  string        
 2   Brand          80238 non-null  string        
 3   Color          80238 non-null  string        
 4   Subcategory    80238 non-null  string        
 5   Category       80238 non-null  string        
 6   CustomerKey    80238 non-null  int64         
 7   Customer Code  80238 non-null  string        
 8   Name           80238 non-null  string        
 9   Education      80238 non-null  string        
 10  Occupation     80238 non-null  string        
 11  Continent      80238 non-null  string        
 12  City           80238 non-null  string        
 13  State          80238 non-null  string        
 14  CountryRegion  80238 non-null  string        
 15  OrderDate      80238 no

In [118]:
print(forecast_data.duplicated().sum())
print(forecast_data.isna().sum()) 

0
CountryRegion    0
Brand            0
Forecast         0
Year             0
dtype: int64


In [119]:
# modeling using star schema
dim_product = Sales_data[['ProductKey', 'Product Name', 'Brand', 'Color', 'Subcategory', 'Category']].drop_duplicates()
dim_customer = Sales_data[['CustomerKey', 'Customer Code', 'Name', 'Education', 'Occupation']].drop_duplicates()
Sales_data['CityState'] = Sales_data['City'].str.strip() + '-' + Sales_data['State'].str.strip()

dim_geography = Sales_data[['CityState', 'CountryRegion', 'Continent']].drop_duplicates()
fact_sales = Sales_data[['OrderDate', 'ProductKey', 'CustomerKey', 'CityState', 'Quantity', 'Net Price']].drop_duplicates()

In [120]:
print(dim_product.duplicated().sum())
print(dim_product.isna().sum()) 
print(dim_customer.duplicated().sum())
print(dim_customer.isna().sum()) 
print(dim_geography.duplicated().sum())
print(dim_geography.isna().sum()) 
print(fact_sales.duplicated().sum())
print(fact_sales.isna().sum()) 

0
ProductKey      0
Product Name    0
Brand           0
Color           0
Subcategory     0
Category        0
dtype: int64
0
CustomerKey      0
Customer Code    0
Name             0
Education        0
Occupation       0
dtype: int64
0
CityState        0
CountryRegion    0
Continent        0
dtype: int64
0
OrderDate      0
ProductKey     0
CustomerKey    0
CityState      0
Quantity       0
Net Price      0
dtype: int64


In [121]:
dim_product.shape

(2495, 6)

In [122]:
dim_customer.shape

(8868, 5)

In [123]:
dim_geography.shape

(306, 3)

In [124]:
fact_sales.shape

(80238, 6)

In [125]:
dim_product.to_csv('dim_product.csv',index=False)
dim_customer.to_csv('dim_customer.csv',index=False)
dim_geography.to_csv('dim_geography.csv',index=False)
fact_sales.to_csv('fact_sales.csv',index=False)
forecast_data.to_csv('fact_forecast.csv',index=False)
